In [19]:
%pip install --upgrade python-dotenv nest_asyncio pydantic google-genai requests pandas

from IPython.display import clear_output ; clear_output()

In [20]:
import os
import random
import asyncio

from dotenv import load_dotenv
import nest_asyncio

from textwrap import dedent
from IPython.display import display, Markdown
def md(str): display(Markdown(str))

from pydantic import BaseModel, Field
from enum import Enum

import pandas as pd

from google import genai
from google.genai import types

load_dotenv()

nest_asyncio.apply()

gemini = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

G25PRO = 'gemini-2.5-pro-preview-03-25'
G25FLASH = 'gemini-2.5-flash-preview-04-17'

In [27]:
class Location(str, Enum):
    NATURE = 'nature'
    OFFICE = 'office'
    PUB = 'pub'
    STREET = 'street'
    HOME = 'home'

class Mood(str, Enum):
    HAPPY = 'happy'
    SAD = 'sad'
    CALM = 'calm'
    SCARY = 'scary'

class TimeOfDay(str, Enum):
    MORNING = 'morning'
    AFTERNOON = 'afternoon'
    EVENING = 'evening'
    NIGHT = 'night'

class Gender(str, Enum):
    MALE = 'male'
    FEMALE = 'female'
    OTHER = 'other'

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    gender: Gender = Field(description="Gender of the person")

class Story(BaseModel):
    people: list[Person] = Field(description="List of people in the story")
    location: Location = Field(description="Location of the story")
    time_of_day: TimeOfDay = Field(description="Time of day in the story")
    mood: Mood = Field(description="Mood of the story")

async def create_story():
    num_people = random.randint(2, 4)
    location = random.choice([loc.value for loc in Location])
    mood = random.choice([mood.value for mood in Mood])
    time_of_day = random.choice([time.value for time in TimeOfDay])

    prompt = dedent(f"""
        - Write a very short story depicting a situation.
        - Describe, the people (with their name, age, gender ({', '.join([gender.value for gender in Gender])}).
        - Describe the location, the time of day, and the mood with sufficient detail,
          to allow the reader to discern them clearly and unambiguously. The reader will be tested
          on their ability to identify the people, location, time of day, and mood from the story,
          so be sure to include enough detail for them to do so.
        - Don't use any special characters or markdown formatting.
        - The story should be roughly 200 tokens long.

        <story-specification>
          - Number of people: {num_people}
          - Location: {location}
          - Time of day: {time_of_day}
          - Mood: {mood}
        </story-specification>
    """)

    response = await gemini.aio.models.generate_content(
        model=G25FLASH,
        contents=prompt,
        config={
            'thinking_config': {'thinking_budget': 1000},
            'max_output_tokens': 1500,
            'temperature': 0.8,
        },
    )

    return prompt, response.text

async def parse_story(story):
    prompt = dedent(f"""
        Parse the story and extract the following information:
        - People: name, age, gender ({', '.join([gender.value for gender in Gender])})
        - Location: {', '.join([loc.value for loc in Location])}
        - Time of day: {', '.join([time.value for time in TimeOfDay])}
        - Mood: {', '.join([mood.value for mood in Mood])}

    <story>
      {story}
    </story>
    """)
    response = await gemini.aio.models.generate_content(
        model=G25FLASH,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': Story,
            'thinking_config': {'thinking_budget': 3000},
            'temperature': 0.3,
        },
    )

    obj = response.parsed
    people = sorted(obj.people, key=lambda x: x.name)
    return dedent(f"""\
    PEOPLE:
    {'\n    '.join([f"  - {person.name}, {person.age}, {person.gender.value}" for person in people])}
    LOCATION: {obj.location.value}
    TIME_OF_DAY: {obj.time_of_day.value}
    MOOD: {obj.mood.value}
    """)

prompt, response = await create_story()
parsed = await parse_story(response)
md(f"### Prompt\n\n```{prompt}```\n\n### Response\n\n{response}\n\n### Parsed\n\n```\n{parsed}\n```")

### Prompt

```
- Write a very short story depicting a situation.
- Describe, the people (with their name, age, gender (male, female, other).
- Describe the location, the time of day, and the mood with sufficient detail,
  to allow the reader to discern them clearly and unambiguously. The reader will be tested
  on their ability to identify the people, location, time of day, and mood from the story,
  so be sure to include enough detail for them to do so.
- Don't use any special characters or markdown formatting.
- The story should be roughly 200 tokens long.

<story-specification>
  - Number of people: 4
  - Location: home
  - Time of day: evening
  - Mood: calm
</story-specification>
```

### Response

The soft glow from a single lamp illuminated the living room. Outside, the windows showed only darkness, confirming it was evening in their house. Sarah, a 45-year-old female, sat on the sofa, knitting quietly. Next to her, David, a 47-year-old male, read a book, the pages turning with soft rustles. Across the room, their daughter Emily, 16, female, sketched in a notebook, a gentle hum sometimes escaping her lips. On the floor nearby, Max, their 10-year-old male son, carefully built a Lego model, concentrating silently. A peaceful quiet filled the space, punctuated only by the gentle sounds of their activities. The mood was one of complete calm and relaxation.

### Parsed

```
PEOPLE:
  - David, 47, male
  - Emily, 16, female
  - Max, 10, male
  - Sarah, 45, female
LOCATION: home
TIME_OF_DAY: evening
MOOD: calm

```

In [25]:
async def a_gen_item():
    _, response = await create_story()
    parsed = await parse_story(response)
    return response, parsed

stories = pd.DataFrame(
    await asyncio.gather(*[a_gen_item() for _ in range(100)]),
    columns=['input', 'output'],
)

display(stories.head(3).style.set_properties(
    **{'text-align': 'left', 'vertical-align': 'top', 'white-space': 'pre-wrap', 'width': '50%'},
))

,input,output
0,"Anya, a sixteen-year-old female, clutched her brother Ben's arm. Ben, seventeen and male, tried to look brave but his eyes darted nervously around the dense woods. Fifteen-year-old female Chloe walked close behind, stumbling slightly on the uneven ground. Long, distorted shadows stretched from the gnarled trees as the weak afternoon sun struggled to pierce the thick canopy. A sudden snap of a twig nearby made them all freeze, hearts pounding. The silence that followed felt heavy and menacing. They were deep in the Whispering Woods, far from any trail, and the air felt cold despite the time of day.","PEOPLE: - Anya, 16, female - Ben, 17, male - Chloe, 15, female LOCATION: nature TIME_OF_DAY: afternoon MOOD: scary"
1,"The afternoon sun streamed through the tall windows of the cozy pub, casting warm stripes across the wooden floorboards. The air smelled faintly of hops and old wood. At a small table near the back sat Sarah, a female aged 35, her face bright with laughter. Opposite her was Tom, a male aged 38, a wide grin fixed on his face as he listened to her story. They clinked their glasses together, a sound lost in the low hum of afternoon conversation around them. The easy smiles and relaxed postures made it clear they were enjoying themselves immensely, completely content in the moment.","PEOPLE: - Sarah, 35, female - Tom, 38, male LOCATION: pub TIME_OF_DAY: afternoon MOOD: happy"
2,"The stale air in the pub hung heavy. Sunlight, weak and watery, slanted through the high windows, illuminating dust motes dancing above the sticky floor. It was unusually quiet for The Dog and Duck, especially at half past ten on a Saturday morning. John, a man of fifty-five years, sat hunched over a pint glass, his face etched with sorrow. Beside him, Mary, fifty-two and female, dabbed at her eyes with a tissue, her shoulders shaking slightly. Across the small table, David, a male just twenty-eight, stared blankly ahead, his youthful face pale and drawn. No one spoke much, the silence amplifying the profound sadness that enveloped the small group.","PEOPLE: - David, 28, male - John, 55, male - Mary, 52, female LOCATION: pub TIME_OF_DAY: morning MOOD: sad"
